In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import pprint
import cv2
import shutil
import os

In [ ]:
train_json = '../Annotations/train.json'
train_data = '/media/arcas/PEN/train/'

Estructura basica de les dades

In [ ]:
with open(train_json, encoding='UTF-8') as json_file:
    data = json.load(json_file)
    pprint.pprint(data[0])

Percentatge d'imatges que no es poden respondre

In [ ]:
answerability = [d["answerable"] for d in data]
non_answerable = len(data) - sum(answerability)
non_answble_imgs_percent = (non_answerable / len(data)) * 100
non_answble_imgs_labels = [d["image"] for d in data if d["answerable"] == 0]

print(f"Non-answerable: {non_answerable} imgs = {round(non_answble_imgs_percent, 2)}%")

In [ ]:
sample_img = non_answble_imgs_labels[20]
img = cv2.imread(train_data + sample_img)
imgc = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.imshow(imgc)
plt.axis('off')
plt.show()

Copiar les imatges NAI (Non-Answerable-Images) en una carpeta a aprt

In [ ]:
destination = '../data/NAI_train/'

In [ ]:
# Creating storing folders
if not (os.path.isdir(destination)):
    os.mkdir(destination)

In [ ]:
if len(os.listdir(destination)) == 0:
    for fs in non_answble_imgs_labels:
        full_file = train_data + fs
        shutil.copy(full_file, destination + fs)
    print("Files moved succesfully!")

# Detect images with text

In [ ]:
import cv2
import pytesseract
 
def find_text(sample_img):
    err_count = 0
    try:
        img = cv2.imread(sample_img)

        # Preprocessing the image starts
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Performing OTSU threshold
        ret, thresh1 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)

        # Specify structure shape and kernel size.
        # Kernel size increases or decreases the area
        # of the rectangle to be detected.
        # A smaller value like (10, 10) will detect
        # each word instead of a sentence.
        rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (18, 18))

        # Applying dilation on the threshold image
        dilation = cv2.dilate(thresh1, rect_kernel, iterations = 1)

        # Finding contours
        contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL,
                                                         cv2.CHAIN_APPROX_NONE)
        # Creating a copy of image
        im2 = img.copy()

        # A text file is created and flushed
        file = open("recognized.txt", "w+")
        file.write("")
        file.close()

        # Looping through the identified contours
        # Then rectangular part is cropped and passed on
        # to pytesseract for extracting text from it
        # Extracted text is then written into the text file
        for cnt in contours:
            x, y, w, h = cv2.boundingRect(cnt)

            # Drawing a rectangle on copied image
            rect = cv2.rectangle(im2, (x, y), (x + w, y + h), (0, 255, 0), 2)

            # Cropping the text block for giving input to OCR
            cropped = im2[y:y + h, x:x + w]

            # Open the file in append mode
            file = open("recognized.txt", "a")

            # Apply OCR on the cropped image
            text = pytesseract.image_to_string(cropped)

            # Appending the text into file
            file.write(text)
            file.write("\n")

            # Close the file
            file.close

        if os.path.getsize("./recognized.txt") > 0:
            return True
        else:
            return False
        
    except:
        err_count += 1

In [ ]:
%%time
# Array containing bool values wheather the images had text or not
results = np.array([find_text(destination + img) 
                    for img in non_answble_imgs_labels], dtype=bool)

# Count how many True values (images containing text)
count = np.count_nonzero(results)
perct_of_text = round(count / results.shape[0] * 100, 2)

print(f"Total number of imgs = {results.shape[0]}\nTotal percentage of imgs containing text = {perct_of_text}%")
print(err_count)